In [20]:
import pandas as pd
import numpy as np
import re

In [13]:
df = pd.read_csv('train.csv')
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [4]:
df['title'] = df['title'].fillna(' ')

In [5]:
df['author'] = df['author'].fillna(' ')

In [6]:
df['text'] = df['text'].fillna(' ')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20800 non-null  object
 2   author  20800 non-null  object
 3   text    20800 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [8]:
from razdel import tokenize
import pymorphy2

In [9]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/work/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
stopword = stopwords.words('english')

In [11]:
print(len(stopword))

179


Честно не писала сама, а взяла из урока

In [27]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)
    
    return text

cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text):    
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист лемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w) > 1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords = [i for i in words_lem if not i in stopword] # [6]
    
    return words_lem_without_stopwords

In [18]:
%%time
from tqdm import tqdm
tqdm.pandas()

CPU times: user 399 µs, sys: 138 µs, total: 537 µs
Wall time: 497 µs


In [21]:
%%time
from tqdm import tqdm
tqdm.pandas()

df['title'] = df['title'].progress_apply(lambda x: clean_text(x))

  0%|          | 0/20800 [00:00<?, ?it/s]<ipython-input-17-898b573b2830>:14: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
100%|██████████| 20800/20800 [00:00<00:00, 24727.61it/s]

CPU times: user 778 ms, sys: 26.3 ms, total: 804 ms
Wall time: 852 ms


In [23]:
df['author'] = df['author'].progress_apply(lambda x: clean_text(x))

100%|██████████| 20800/20800 [00:00<00:00, 68333.61it/s]


In [24]:
df['text'] = df['text'].progress_apply(lambda x: clean_text(x))

100%|██████████| 20800/20800 [00:33<00:00, 622.79it/s]


In [28]:
df['title'] = df['title'].progress_apply(lambda x: lemmatization(x))

100%|██████████| 20800/20800 [00:04<00:00, 4305.12it/s]


In [29]:
df['author'] = df['author'].progress_apply(lambda x: lemmatization(x))

100%|██████████| 20800/20800 [00:01<00:00, 18084.55it/s]


In [30]:
df['text'] = df['text'].progress_apply(lambda x: lemmatization(x))

100%|██████████| 20800/20800 [04:07<00:00, 84.11it/s] 


In [31]:
df

,id,title,author,text,label
0,0,"[house, dem, aide, eve, see, comey, letter, ti...","[darrell, lucus]","[house, dem, aide, eve, see, comey, letter, ti...",1
1,1,"[fly, hillary, cli, big, woma, campus, breitbart]","[da, iel, fly]","[ever, get, feeli, life, circles, rou, dabout,...",0
2,2,"[truth, might, get, fired]","[co, sortium, ewscom]","[truth, might, get, fired, october, te, sio, b...",1
3,3,"[civilia, killed, si, gle, us, airstrike, bee,...","[jessica, purkiss]","[videos, civilia, killed, si, gle, us, airstri...",1
4,4,"[ira, ia, woma, jailed, fictio, al, published,...","[howard, port, oy]","[pri, ira, ia, woma, bee, se, te, ced, six, ye...",1
...,...,...,...,...,...
20795,20795,"[rapper, ti, trump, poster, child, white, supr...","[jerome, hudso]","[rapper, loaded, black, celebrities, met, ald,...",0
20796,20796,"[fl, playoffs, schedule, matchups, odds, ew, y...","[jami, hoffma]","[whe, gree, bay, packers, lost, washi, gto, re...",0
20797,20797,"[macy, said, receive, takeover, approach, huds...","[michael, de, la, merced, rachel, abrams]","[macy, today, grew, io, several, great, ames, ...",0
20798,20798,"[ato, russia, hold, parallel, exercises, balka]","[alex, sary]","[ato, russia, hold, parallel, exercises, balka...",1
